## Using OSL Batch Utils (currently error with extract_fiducials_from_fif)

In [56]:
import os
import pathlib
from glob import glob
from dask.distributed import Client
import coinsmeg_data as coinsmeg
from osl import source_recon, utils
import numpy as np

# Directories
BASE_DIR = '/ohba/pi/lhunt/datasets/coins-meg_data'
DATA_DIR = coinsmeg.RAW_DIR # this is the same as BASE_DIR as raw data is stored in the base directory
PREPROC_DIR = coinsmeg.PREPROCESSED_DIR
PREPROC_FILE = PREPROC_DIR + "/sub-{0}_ses-2-meg_task-coinsmeg_run-{1}_meg_transsss/sub-{0}_ses-2-meg_task-coinsmeg_run-{1}_meg_transsss_preproc_raw.fif"

PREPROC_OUT_TEST = PREPROC_DIR + "/preprocessed-test/"

ANAT_DIR = coinsmeg.get_sub_anat_dir("sub-{0}")
SMRI_FILE = ANAT_DIR + "/sub-{0}_T1w.nii"

FSL_DIR = "/home/ali/fsl"


In [49]:
# define needed functions
def fix_headshape_points(outdir, subject):
    filenames = source_recon.rhino.get_coreg_filenames(outdir, subject)

    # Load saved headshape and nasion files
    hs = np.loadtxt(filenames["polhemus_headshape_file"])
    nas = np.loadtxt(filenames["polhemus_nasion_file"])
    lpa = np.loadtxt(filenames["polhemus_lpa_file"])
    rpa = np.loadtxt(filenames["polhemus_rpa_file"])

    # Remove headshape points on the nose
    remove = np.logical_and(hs[1] > max(lpa[1], rpa[1]), hs[2] < nas[2])
    hs = hs[:, ~remove]

    # Overwrite headshape file
    utils.logger.log_or_print(f"overwritting {filenames['polhemus_headshape_file']}")
    np.savetxt(filenames["polhemus_headshape_file"], hs)

In [10]:
utils.logger.set_up(level="INFO")
source_recon.setup_fsl(FSL_DIR)

 OSL osl_logger: handler 'console' level set to 'INFO'
 OSL Logger Started


In [43]:
# Get subjects
subs = []

data_sub_folders = sorted(filter(lambda path: "sub-" in path, glob(DATA_DIR + '/*'))) # returns a list of paths e.g., '/ohba/pi/lhunt/datasets/coins-meg_data/sub-22'

for subject in data_sub_folders:
    subs.append(pathlib.Path(subject).stem) # subs is now a list of ['sub-01', 'sub-02', ...]

# Setup files
smri_files = []
preproc_files = []
subjects = []
for subject in subs:
    sub = subject.split('-')[1] # e.g., '01'
    mri_file = SMRI_FILE.format(sub) # e.g., '/ohba/pi/lhunt/datasets/coins-meg_data/sub-01/ses-3-structural/anatsub-01_T1w.nii'
    if not os.path.exists(mri_file): # if the smri file for subject, specified by mri_file, does not actually exist...
        continue
    else:
        smri_files.append(mri_file) # add the smri filepath to the list smri_files
    preproc_file = glob(PREPROC_FILE.replace('{0}', sub).replace('{1}', '*')) # list of 4 ...transsss_preproc_raw.fif files
    # maybe exclude sub 21 for now as they have 5 fif runs
    for f in preproc_file:
        subjects.append(f"{subject}_run-{f.split('run-')[1][0]}")
        preproc_files.append(f)

In [46]:
print(preproc_files)

['/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-04_ses-2-meg_task-coinsmeg_run-2_meg_transsss/sub-04_ses-2-meg_task-coinsmeg_run-2_meg_transsss_preproc_raw.fif', '/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-04_ses-2-meg_task-coinsmeg_run-3_meg_transsss/sub-04_ses-2-meg_task-coinsmeg_run-3_meg_transsss_preproc_raw.fif', '/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-04_ses-2-meg_task-coinsmeg_run-1_meg_transsss/sub-04_ses-2-meg_task-coinsmeg_run-1_meg_transsss_preproc_raw.fif', '/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-04_ses-2-meg_task-coinsmeg_run-4_meg_transsss/sub-04_ses-2-meg_task-coinsmeg_run-4_meg_transsss_preproc_raw.fif', '/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-07_ses-2-meg_task-coinsmeg_run-4_meg_transsss/sub-07_ses-2-meg_task-coinsmeg_run-4_meg_transsss_preproc_raw.fif', '/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/sub-07_ses-2-meg_ta

In [47]:
print(smri_files)

['/ohba/pi/lhunt/datasets/coins-meg_data/sub-04/ses-3-structural/anat/sub-04_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-05/ses-3-structural/anat/sub-05_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-06/ses-3-structural/anat/sub-06_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-07/ses-3-structural/anat/sub-07_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-09/ses-3-structural/anat/sub-09_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-10/ses-3-structural/anat/sub-10_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-11/ses-3-structural/anat/sub-11_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-12/ses-3-structural/anat/sub-12_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-13/ses-3-structural/anat/sub-13_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-14/ses-3-structural/anat/sub-14_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-15/ses-3-structural/anat/sub-15_T1w.nii', '/ohba/pi/lhunt/datasets/coins-meg_data/sub-16/ses-3-

In [64]:
# test case on two subjects
if __name__ == "__main__":
    utils.logger.set_up(level="INFO")
    source_recon.setup_fsl(FSL_DIR)

    # Get subjects
    #subs = []
    #for subject in sorted(glob(ANAT_DIR + '/*')):
    #    subs.append(pathlib.Path(subject).stem)

    subs = ['sub-04']
    
    # Setup files
    smri_files = []
    preproc_files = []
    subjects = []
    for subject in subs:
        sub = subject.split('-')[1] # e.g., '01'
        mri_file = SMRI_FILE.format(sub) # e.g., '/ohba/pi/lhunt/datasets/coins-meg_data/sub-01/ses-3-structural/anatsub-01_T1w.nii'
        if not os.path.exists(mri_file): # if the smri file for subject, specified by mri_file, does not actually exist...
            continue
        preproc_file = glob(PREPROC_FILE.replace('{0}', sub).replace('{1}', '*')) # list of 4 ...transsss_preproc_raw.fif files
        # maybe exclude sub 21 for now as they have 5 fif runs
        for f in preproc_file:
            subjects.append(f"{subject}_run-{f.split('run-')[1][0]}")
            preproc_files.append(f)
            smri_files.append(mri_file) # add the smri filepath to the list smri_files

    # Settings
    config = """
        source_recon:
        - extract_fiducials_from_fif: {}
        - fix_headshape_points: {}
        - compute_surfaces:
            include_nose: True
        - coregister:
            use_nose: True
            use_headshape: True
        - forward_model:
            model: Single Layer
        - beamform_and_parcellate:
            freq_range: [1, 80]
            chantypes: [mag, grad]
            rank: {meg: 60}
            parcellation_file: HarvOxf-sub-Schaefer100-combined-2mm_4d_ds8.nii.gz
            method: spatial_basis
            orthogonalisation: symmetric
            extra_chans: [eog, ecg]
    """

    # Setup parallel processing
    client = Client(n_workers=8, threads_per_worker=1)

    # Run beamforming and parcellation
    source_recon.run_src_batch(
        config,
        outdir=PREPROC_OUT_TEST,
        subjects=subjects,
        preproc_files=preproc_files,
        smri_files=smri_files,
        extra_funcs=[fix_headshape_points],
        dask_client=True,
    )


 OSL osl_logger: handler 'console' level set to 'INFO'
 OSL Logger Started
 OSL osl_logger: handler 'console' level set to 'INFO'
 logging to file: /ohba/pi/lhunt/datasets/coins-meg_data/derivatives/preprocessed/preprocessed-test/logs/osl_batch.log
 Starting OSL Batch Source Reconstruction
 Running config
 {'source_recon': [{'extract_fiducials_from_fif': {}},
                  {'fix_headshape_points': {}},
                  {'compute_surfaces': {'include_nose': True}},
                  {'coregister': {'use_headshape': True, 'use_nose': True}},
                  {'forward_model': {'model': 'Single Layer'}},
                  {'beamform_and_parcellate': {'chantypes': ['mag', 'grad'],
                                               'extra_chans': ['eog', 'ecg'],
                                               'freq_range': [1, 80],
                                               'method': 'spatial_basis',
                                               'orthogonalisation': 'symmetric',
     

/home/ali/Desktop/osl/osl/source_recon/batch.py:153: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.utils._logging.set_log_file(logfile)
  File "/home/ali/Desktop/osl/osl/source_recon/batch.py", line 199, in run_src_chain
    wrapped_func(
  File "/home/ali/Desktop/osl/osl/source_recon/batch.py", line 191, in wrapped_func
    args_with_defaults = args[-len(defaults):]
/home/ali/Desktop/osl/osl/source_recon/batch.py:153: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.utils._logging.set_log_file(logfile)
  File "/home/ali/Desktop/osl/osl/source_recon/batch.py", line 199, in run_src_chain
    wrapped_func(
  File "/home/ali/Desktop/osl/osl/source_recon/batch.py", line 191, in wrapped_func
    args_with_defaults = args[-len(defaults):]
/home/ali/Desktop/osl/osl/source_recon/batch.py:153: RuntimeWarning: Log entries will be appended to the file.

## NOT Using OSL Batch Utils -- This is working!

In [70]:
import re
import os
import os.path as op
import osl
from pprint import pprint
from osl import utils
from osl import source_recon
import numpy as np
import coinsmeg_data as coinsmeg
import open3d
from IPython.display import HTML, display
import mne
import warnings


In [67]:
# functions
def copy_polhemus_files(recon_dir, subject, preproc_file, smri_file, logger):
    polhemus_headshape = np.loadtxt(op.join(polhemus_dir, 'polhemus_headshape.txt'))
    polhemus_nasion = np.loadtxt(op.join(polhemus_dir, 'polhemus_nasion.txt'))
    polhemus_rpa = np.loadtxt(op.join(polhemus_dir, 'polhemus_rpa.txt'))
    polhemus_lpa = np.loadtxt(op.join(polhemus_dir, 'polhemus_lpa.txt'))
    
    #  Get coreg filenames
    filenames = source_recon.rhino.get_coreg_filenames(recon_dir, subject)

    # Save
    np.savetxt(filenames["polhemus_nasion_file"], polhemus_nasion)
    np.savetxt(filenames["polhemus_rpa_file"], polhemus_rpa)
    np.savetxt(filenames["polhemus_lpa_file"], polhemus_lpa)
    np.savetxt(filenames["polhemus_headshape_file"], polhemus_headshape)


In [61]:
# Directories
preproc_dir = op.join(coinsmeg.DERIVATIVES_DIR, "preprocessed")
recon_dir = op.join(coinsmeg.DERIVATIVES_DIR, "recon")

In [65]:
# Get subjects
subs = []

data_sub_folders = sorted(filter(lambda path: "sub-" in path, glob(DATA_DIR + '/*'))) # returns a list of paths e.g., '/ohba/pi/lhunt/datasets/coins-meg_data/sub-22'

for subject in data_sub_folders:
    subs.append(pathlib.Path(subject).stem) # subs is now a list of ['sub-01', 'sub-02', ...]

# Create a list with '_run-X' appended for each subject
sub_run_combos = []
for sub in subs:
    for run in range(1, 5):  # Loop from 1 to 4
        sub_run_combos.append(f"{sub}_run-{run}") # sub_run_combos is now a list of ['sub-01_run-1', 'sub-01_run-2', ...]

print(sub_run_combos)

['sub-01_run-1', 'sub-01_run-2', 'sub-01_run-3', 'sub-01_run-4', 'sub-02_run-1', 'sub-02_run-2', 'sub-02_run-3', 'sub-02_run-4', 'sub-03_run-1', 'sub-03_run-2', 'sub-03_run-3', 'sub-03_run-4', 'sub-04_run-1', 'sub-04_run-2', 'sub-04_run-3', 'sub-04_run-4', 'sub-05_run-1', 'sub-05_run-2', 'sub-05_run-3', 'sub-05_run-4', 'sub-06_run-1', 'sub-06_run-2', 'sub-06_run-3', 'sub-06_run-4', 'sub-07_run-1', 'sub-07_run-2', 'sub-07_run-3', 'sub-07_run-4', 'sub-08_run-1', 'sub-08_run-2', 'sub-08_run-3', 'sub-08_run-4', 'sub-09_run-1', 'sub-09_run-2', 'sub-09_run-3', 'sub-09_run-4', 'sub-10_run-1', 'sub-10_run-2', 'sub-10_run-3', 'sub-10_run-4', 'sub-11_run-1', 'sub-11_run-2', 'sub-11_run-3', 'sub-11_run-4', 'sub-12_run-1', 'sub-12_run-2', 'sub-12_run-3', 'sub-12_run-4', 'sub-13_run-1', 'sub-13_run-2', 'sub-13_run-3', 'sub-13_run-4', 'sub-14_run-1', 'sub-14_run-2', 'sub-14_run-3', 'sub-14_run-4', 'sub-15_run-1', 'sub-15_run-2', 'sub-15_run-3', 'sub-15_run-4', 'sub-16_run-1', 'sub-16_run-2', 'sub-16

In [77]:
# test with only two sub/run combos
sub_run_combos =['sub-04_run-1','sub-04_run-2']

In [80]:
for sub_run_combo in sub_run_combos:

    # extract the subject and run id
    subject_id, run_id = sub_run_combo.split('_') # will produce subject_id = 'sub-XX', run_id = 'run-X'
    # set paths for where to look for necessary files
    anat_dir = coinsmeg.get_sub_anat_dir(subject_id)
    smri_file = f"{anat_dir}/{subject_id}_T1w.nii"
    fif_file = f'{preproc_dir}/{subject_id}_ses-2-meg_task-coinsmeg_{run_id}_meg_transsss/{subject_id}_ses-2-meg_task-coinsmeg_{run_id}_meg_transsss_preproc_raw.fif'
    
    if not os.path.exists(smri_file):
        print(f"WARNING: smri_file does not exist for {sub_run_combo}!")
        continue # skip over the rest of the code for this sub_run_combo

    source_recon.rhino.compute_surfaces(
        smri_file,
        recon_dir,
        sub_run_combo,
        include_nose=True,
    )
    # check in fsleyes
    source_recon.rhino.surfaces_display(recon_dir, sub_run_combo)
    
    polhemus_dir = op.join(recon_dir, sub_run_combo, "polhemus")
    # make directory if it doesn't yet exist
    os.makedirs(polhemus_dir, exist_ok=True)

    from osl.source_recon.rhino.polhemus import extract_polhemus_from_info

    extract_polhemus_from_info(
        fif_file = fif_file,
        headshape_outfile=op.join(polhemus_dir, "polhemus_headshape.txt"),
        nasion_outfile=op.join(polhemus_dir, "polhemus_nasion.txt"),
        rpa_outfile=op.join(polhemus_dir, "polhemus_rpa.txt"),
        lpa_outfile=op.join(polhemus_dir, "polhemus_lpa.txt")
    )

    # recall that our maxfiltering options were
    # f"--maxpath /neuro/bin/util/maxfilter --mode multistage --scanner Neo --tsss --headpos --movecomp --trans {trans_file}",)
            #  the maxfilter aligns all runs of a participant so that the head position is the same within each participant
    
    copy_polhemus_files(recon_dir, sub_run_combo, [], [], [])

    # Then we run the coreg, for real.

    source_recon.rhino.coreg(
        fif_file, # full path to the MNE raw fif file
        recon_dir, # full path to the directory that contains the subject directories RHINO outputs
        sub_run_combo, # the name of the subject directories RHINO outputs to
        use_headshape=True,     #use the headshape points to refine the coregistration?
        use_nose=True, # use the nose headshape points to refine the coregistration?
    )

    # now view result
    source_recon.rhino.coreg_display(subjects_dir = "/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon", 
                                    subject = sub_run_combo,
                                    filename = f"{recon_dir}/{sub_run_combo}/rhino/coreg/coreg_display_plot.html") # saves an interactive html plot at this location

    # Compute forward model
    gridstep = 10
    source_recon.rhino.forward_model(
        recon_dir,
        sub_run_combo,
        model="Single Layer",
        gridstep=gridstep,
    )

    # view results
    source_recon.rhino.bem_display(
        recon_dir,
        sub_run_combo,
        display_outskin_with_nose=False,
        display_sensors=True,
        plot_type="surf",
        filename=f"{recon_dir}/{sub_run_combo}/rhino/coreg/bem_display_plot.html",
    )

    from mne import read_forward_solution

    # load forward solution
    fwd_fname = op.join(recon_dir, sub_run_combo, "rhino", "model-fwd.fif") 
    # tutorial said source_recon.rhino.get_coreg_filenames(recon_dir, subjects[0])["forward_model_file"]
    # but this did not return a match for "forward_model_file"
    print(fwd_fname)

    fwd = read_forward_solution(fwd_fname)
    leadfield = fwd["sol"]["data"]
    print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

    import mne

    # Temporal filtering

    chantypes = ["grad", "mag"] 

    # Get and setup the data
    data = mne.io.read_raw_fif(fif_file, preload=True)
    data = data.pick(chantypes)

    # Filter to the beta band
    print("Temporal Filtering")
    data = data.filter(
        l_freq=1,
        h_freq=30,
        method="iir",
        iir_params={"order": 5, "btype": "bandpass", "ftype": "butter"},
    )
    print("Completed")

    # Compute BEAMFORMER WEIGHTS
    from osl.source_recon import rhino, beamforming, parcellation
      
    # Make LCMV beamformer filters
    # Note that this will exclude any bad time segments when calculating the beamformer filters
    filters = beamforming.make_lcmv(
        recon_dir,
        sub_run_combo,
        data,
        chantypes,
        pick_ori="max-power-pre-weight-norm",
        rank={"meg": 60},
    )

    print("Applying beamformer spatial filters")

    # stc is source space time series (in head/polhemus space).
    stc = beamforming.apply_lcmv(data, filters)

    # Convert from head/polhemus space to standard brain grid in MNI space
    recon_timeseries_mni, reference_brain_fname, recon_coords_mni, _ = \
            beamforming.transform_recon_timeseries(recon_dir, 
                                                    sub_run_combo, 
                                                    recon_timeseries=stc.data, 
                                                    reference_brain="mni")

    print("Completed")
    print("Dimensions of reconstructed timeseries in MNI space is (dipoles x all_tpts) = {}".format(recon_timeseries_mni.shape))

    # PARCELLATION
    parcellation_fname = 'HarvOxf-sub-Schaefer100-combined-2mm_4d_ds8.nii.gz'

    print("Parcellating data")

    # Apply parcellation to (voxels x all_tpts) data contained in recon_timeseries_mni.
    # The resulting parcel_timeseries will be (parcels x all_tpts) in MNI space
    # where all_tpts includes bad time segments
    parcel_ts, _, _ = parcellation.parcellate_timeseries(
        parcellation_fname, # corresponds to the -beamform_and_parcellate: method: parcellation_file
        recon_timeseries_mni, # reconstructed timeseries in MNI space; dimensions are (dipoles x all_tpts)
        recon_coords_mni,  # dimensions are 3 x dipoles ((3, 2527)). the 3 rows are x, y, and z coordinates in MNI space
        "spatial_basis",  # corresponds to the -beamform_and_parcellate: method: setting
        recon_dir,
    )

    print("Completed")
    print("Dimensions of parcel timeseries in MNI space is (nparcels x all_tpts) = {}".format(parcel_ts.shape))

    # Create mne raw object for the parcellated data

    # We reload raw data to ensure that the stim channel is in there
    raw = mne.io.read_raw_fif(fif_file) # recall that fif_file is a specific subject/run
    parc_raw = parcellation.convert2mne_raw(parcel_ts, raw)

    print("Dimensions of parc_raw are (nparcels x all_tpts) = {}".format(parc_raw.get_data().shape))

    # source space data directory
    src_dir = op.join(coinsmeg.DERIVATIVES_DIR, "src", sub_run_combo) # directory for saving source reconstructed files
    os.makedirs(src_dir, exist_ok=True)

    # save parc_raw into the src_dir
    parc_raw.save(op.join(src_dir, "parc_raw.fif"), overwrite=True)

 *** RUNNING OSL RHINO COMPUTE SURFACES ***
 The nose is going to be added to the outer skin (scalp) surface.
 Please ensure that the structural MRI has a FOV that includes the nose
 reorienting subject brain to be RADIOLOGICAL
 You can use the following call to check the passed in structural MRI is appropriate,
 including checking that the L-R, S-I, A-P labels are sensible:
 In Python:
 fsleyes("/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-04_run-1/rhino/surfaces/smri.nii.gz", "/home/ali/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz")
 From the cmd line:
 fsleyes /ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-04_run-1/rhino/surfaces/smri.nii.gz /home/ali/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz
 Running BET pre-FLIRT...
 Running FLIRT...
 Running BET and BETSURF...
 Refining scalp surface...
 Adding nose to scalp surface...
 rhino.surfaces.surfaces_display("/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon", "sub-04_run-1") can be used to check 

 *** RUNNING OSL RHINO FORWARD MODEL ***
 Using bet_inskull_surf_file for single shell surface
 *** OSL RHINO FORWARD MODEL COMPLETE ***
 BEM surface: number of dipoles = 1222


/home/ali/anaconda3/envs/osl/lib/python3.8/site-packages/pyvista/plotting/renderer.py:536: UserWarning: VTK compiled with OSMesa does not properly support FXAA anti-aliasing and SSAA will be used instead.
  warnings.warn(


 saving /ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-04_run-1/rhino/coreg/bem_display_plot.html
/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-04_run-1/rhino/model-fwd.fif
Leadfield size : 306 sensors x 3666 dipoles
Temporal Filtering
Completed
 *** RUNNING OSL MAKE LCMV ***
 variance for chantype grad is 1.705213409394786e-23
 variance for chantype mag is 3.780413292790151e-26
 *** OSL MAKE LCMV COMPLETE ***
Applying beamformer spatial filters
 beamforming.apply_lcmv
 spatial_resolution = 10 mm
Completed
Dimensions of reconstructed timeseries in MNI space is (dipoles x all_tpts) = (2527, 794101)
Parcellating data
 gridstep = 10 mm


Completed
Dimensions of parcel timeseries in MNI space is (nparcels x all_tpts) = (114, 794101)
Dimensions of parc_raw are (nparcels x all_tpts) = (115, 853000)
 *** RUNNING OSL RHINO COMPUTE SURFACES ***
 The nose is going to be added to the outer skin (scalp) surface.
 Please ensure that the structural MRI has a FOV that includes the nose
 reorienting subject brain to be RADIOLOGICAL
 You can use the following call to check the passed in structural MRI is appropriate,
 including checking that the L-R, S-I, A-P labels are sensible:
 In Python:
 fsleyes("/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-04_run-2/rhino/surfaces/smri.nii.gz", "/home/ali/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz")
 From the cmd line:
 fsleyes /ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-04_run-2/rhino/surfaces/smri.nii.gz /home/ali/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz
 Running BET pre-FLIRT...
 Running FLIRT...
 Running BET and BETSURF...
 Refining scalp surface...
 A

/home/ali/anaconda3/envs/osl/lib/python3.8/site-packages/pyvista/plotting/renderer.py:536: UserWarning: VTK compiled with OSMesa does not properly support FXAA anti-aliasing and SSAA will be used instead.
  warnings.warn(


 saving /ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-04_run-2/rhino/coreg/bem_display_plot.html
/ohba/pi/lhunt/datasets/coins-meg_data/derivatives/recon/sub-04_run-2/rhino/model-fwd.fif
Leadfield size : 306 sensors x 3666 dipoles
Temporal Filtering
Completed
 *** RUNNING OSL MAKE LCMV ***
 variance for chantype grad is 1.945768377782321e-23
 variance for chantype mag is 4.207976302326424e-26
 *** OSL MAKE LCMV COMPLETE ***
Applying beamformer spatial filters
 beamforming.apply_lcmv
 spatial_resolution = 10 mm
Completed
Dimensions of reconstructed timeseries in MNI space is (dipoles x all_tpts) = (2527, 740501)
Parcellating data
 gridstep = 10 mm


Completed
Dimensions of parcel timeseries in MNI space is (nparcels x all_tpts) = (114, 740501)
Dimensions of parc_raw are (nparcels x all_tpts) = (115, 800000)
